<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/transformers_asr_pipeline_optimisations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A walkthrough of the pleathora of optimisations and nifty features offered via `pipeline` for Automatic Speech Recognition! 🤗

from yours truly - [Vaibhav (VB) Srivastav](https://twitter.com/reach_vb) 🤙

In [1]:
!pip install -q transformers accelerate datasets
!pip install -q ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00


In [2]:
%load_ext autotime

time: 314 µs (started: 2023-09-11 13:50:00 +00:00)


In [5]:
import torch
from transformers import pipeline

time: 8.77 s (started: 2023-09-11 13:50:07 +00:00)


In [6]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-base.en",
                device="cuda:0")

time: 6 s (started: 2023-09-11 13:50:17 +00:00)


## Download a test file!

This is a 2 hour long audio snippet from a revdotcoms Earnings22 dataset.

In [8]:
!wget https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3

--2023-09-11 13:50:30--  https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3 [following]
--2023-09-11 13:50:30--  https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29628186 (28M) [audio/mpeg]
Saving to: ‘4469669.mp3’

4469669.mp3         100%[===================>]  28.25M   168MB/s    in 0.2s    

2023-09-11 13:50:31 (168 MB/s) - ‘4469669.mp3’ s

## Inference

By default Whisper is only capable of running inference for 30 second audios max.

In [9]:
pipe("4469669.mp3")

{'text': " Now it's time. May I start the presentation on transforming Toshiba to enhance your risk value and FY21 second quarter consolidated business results. We are organizing this presentation on online basis. From 4 to 5 o'clock we will be presenting from our side and followed by 30 minutes question session for the middle of the media. The questions from Alice and"}

time: 25.2 s (started: 2023-09-11 13:51:14 +00:00)


# Chunking

In [11]:
pipe("4469669.mp3", chunk_length_s=20)

{'text': " Now, it's time. May I start the presentation on Transforming Toshi Bata Enhanced Heroes Value and FY21 Second Quarter Consulated Business Results. We are organizing this presentation on online basis. From 4 to 5 o'clock, we will be presenting from our side and followed by 30 minutes question session for the middle of the media. The questions from Alice and investors will be accepted from 5.30 to 6 o'clock Japan time. Please be aware of that. Now, we will be collecting questions via telephone conferencing system. As is informed to you beforehand, the conference call system will require the pre-registration beforehand. Let me introduce the presenter today. President NCEO, Satoshi Tsunakawa. Corporate Senior Executive Vice President, Mamoru Hatazawa. Representative Jesus Officer, Corporate Executive Vice President, N C F O, Masayoshi Hirata. We have a chairperson of the strategic review committee outside director, Paul Broth. He is joining from Hong Kong online. My name is Hara

time: 4min 37s (started: 2023-09-11 13:52:56 +00:00)


In [12]:
mem_report()

CPU RAM Free: 10.7 GB
GPU 0 ... Mem Free: 14498MB / 15360MB | Utilization   4%
time: 30.9 ms (started: 2023-09-11 14:00:31 +00:00)


It took about 4 minutes and 37 seconds to run inference on a 2 hour long snippet.

Can we make this faster?

## Cue.. Batching

In [15]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)

{'text': " Now, it's time. May I start the presentation on Transforming Toshi Bata Enhanced Heroes Value and FY21 Second Quarter Consulated Business Results. We are organizing this presentation on online basis. From 4 to 5 o'clock, we will be presenting from our side and followed by 30 minutes question session for the middle of the media. The questions from Alice and investors will be accepted from 5.30 to 6 o'clock Japan time. Please be aware of that. Now, we will be collecting questions via telephone conferencing system. As is informed to you beforehand, the conference call system will require the pre-registration beforehand. Let me introduce the presenter today. President NCEO, Satoshi Tsunakawa. Corporate Senior Executive Vice President, Mamoru Hatazawa. Representative Jesus Officer, Corporate Executive Vice President, N C F O, Masayoshi Hirata. We have a chairperson of the strategic review committee outside director, Paul Broth. He is joining from Hong Kong online. My name is Hara

time: 2min 6s (started: 2023-09-11 14:04:20 +00:00)


Voila!! It only took the `pipeline` 2 minutes and 6 seconds to run inference this time. We were able to reduce the inference time by 50%. 🚀

But, the real question is - can we make it even faster? 💨

## Half - Precision

In [19]:
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-base.en",
                torch_dtype=torch.float16,
                device="cuda:0")

time: 8.95 s (started: 2023-09-11 14:11:34 +00:00)


In [20]:
pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=8)

{'text': " Now, it's time. May I start the presentation on Transforming Toshi Bata Enhanced Heroes Value and FY21 Second Quarter Consulated Business Results. We are organizing this presentation on online basis. From 4 to 5 o'clock, we will be presenting from our side and followed by 30 minutes question session for the middle of the media. The questions from Alice and investors will be accepted from 5.30 to 6 o'clock Japan time. Please be aware of that. Now, we will be collecting questions via telephone conferencing system. As is informed to you beforehand, the conference call system will require the pre-registration beforehand. Let me introduce the presenter today. President NCEO, Satoshi Tsunakawa. Corporate Senior Executive Vice President, Mamoru Hatazawa. Representative Jesus Officer, Corporate Executive Vice President, N C F O, Masayoshi Hirata. We have a chairperson of the strategic review committee outside director, Paul Broth. He is joining from Hong Kong online. My name is Hara

time: 1min 55s (started: 2023-09-11 14:12:36 +00:00)


Ha! Not much, but still about 5% faster at 1 minute and 55 seconds.